In [ ]:
# source: https://github.com/karmus89/blockchain-a-z
!pip install Faker
import datetime
import hashlib
import json
import sys
from faker import Faker

class Blockchain:

    def __init__(self):
        
        self.chain = []
        self.create_block(proof=1, previous_hash='0')

    def create_block(self, proof, previous_hash):
        "Create a block."
        fake = Faker(locale='en_US')
        record = fake.simple_profile()
        block = {
            'index': len(self.chain),
            'timestamp': str(datetime.datetime.now()),
            'proof': proof,
            'previous_hash': previous_hash,
            'record': record,
        }
        self.chain.append(block)
        
        return block

    def get_previous_block(self):
        
        return self.chain[-1]

    def hash_proof(self, previous_proof, next_proof):
        "Calculate the SHA256-hash"
        
        return (hashlib
                .sha256(str(next_proof**2 - previous_proof**2).encode())
                .hexdigest())

    def hash_block(self, block):
        "Calculate the SHA256-hash for a block."

        encoded_block = json.dumps(block, sort_keys=True).encode()
        
        return hashlib.sha256(encoded_block).hexdigest()

    def proof_of_work(self, previous_proof):
        "Calculate a new proof related to the previous block."

        next_proof = 1
        check_proof = False

        while check_proof is False:

            if self.hash_proof(previous_proof, next_proof)[:4] == '0000':
                check_proof = True

            else:
                next_proof += 1

        return next_proof

    def is_chain_valid(self, chain):
        "Validate that block and proof hashes are correct across the chain."

        for i in range(len(chain)):

            if i == 0:
                continue

            if chain[i]['previous_hash'] != self.hash_block(chain[i-1]):
                return False

            previous_proof = chain[i-1]['proof']
            next_proof = chain[i]['proof']

            if self.hash_proof(previous_proof, next_proof)[:4] != '0000':
                
                return False

        return True

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import threading
import requests

from flask import Flask, request, jsonify
from werkzeug.serving import run_simple


class BlockchainApp:

    def __init__(self, host='localhost', port=5000, chain=Blockchain):

        self.host = host
        self.port = port
        self.chain = chain()

        self.host_url = f'http://{self.host}:{self.port}/'
        
        self.app = Flask(__name__)
        self.add_api_endpoints()
        
        self.thread = threading.Thread(
            target=run_simple,
            kwargs={
                'hostname': self.host,
                'port': self.port,
                'application': self.app}
        )

    def __enter__(self):

        self.start()

        return self

    def __exit__(self, *args):

        self.stop()
        
    def add_api_endpoints(self):
        "Add API endpoints to the Flask WebApp."
        
        self.app.add_url_rule(
            rule='/blocks',
            view_func=self.api_blocks,
            methods=['GET', 'POST']
        )
        #5.d
        self.app.add_url_rule(
            rule='/blocks/validate',
            view_func=self.api_validate,
        )
        self.app.add_url_rule(
            rule='/shutdown',
            view_func=self.api_shutdown,
        )
        #5.a
        self.app.add_url_rule(
            rule='/mine_blocks/<int:n_blocks>',
            view_func=self.api_mine_blocks,
            methods=['GET']
        )
        #5.b
        self.app.add_url_rule(
            rule='/male_names',
            view_func=self.api_male_names,
            methods=['GET']
        )
        #5.c
        self.app.add_url_rule(
            rule='/female_names',
            view_func=self.api_female_names,
            methods=['GET']
        )
        #5.e
        self.app.add_url_rule(
            rule='/overwrite_block',
            view_func=self.api_overwrite_block,
            methods=['POST']
        )

    def api_blocks(self):
        "Either retrieve the node's current chain or post a new block to the chain."

        if request.method == 'POST':

            prev_block = self.chain.get_previous_block()
            prev_hash = self.chain.hash_block(prev_block)
            prev_proof = prev_block['proof']

            proof = self.chain.proof_of_work(prev_proof)

            block = self.chain.create_block(proof, prev_hash)

            response = {'message': 'Congratulations, you just mined a Block!',
                        'block': block}

            return (jsonify(response), 200)

        if request.method == 'GET':

            response = {'chain': self.chain.chain,
                        'length': len(self.chain.chain)}

            return (jsonify(response), 200)      

    #5.a
    def api_mine_blocks(self, n_blocks):
        "Mine the specified number of blocks and return the updated blockchain."

        for _ in range(n_blocks):

            prev_block = self.chain.get_previous_block()
            prev_hash = self.chain.hash_block(prev_block)
            prev_proof = prev_block['proof']

            proof = self.chain.proof_of_work(prev_proof)

            block = self.chain.create_block(proof, prev_hash)

        response = {'chain': self.chain.chain,
                    'length': len(self.chain.chain)}

        return (jsonify(response), 200)

    #5.b
    def api_male_names(self):
      "Retrieve all male names from the blockchain."

      male_names = []
      for block in self.chain.chain:
          if 'record' in block:
              if block['record']['sex'] == 'M':
                  male_names.append(block['record']['name'])

      response = {'male_names': male_names}

      return (jsonify(response), 200)



    #5.c
    def api_female_names(self):
      "Retrieve all female names from the blockchain."
      
      female_names = []
      for block in self.chain.chain:
          if 'record' in block:
              if block['record']['sex'] == 'F':
                  female_names.append(block['record']['username'])
      
      response = {'female_names': female_names}
      
      return (jsonify(response), 200)

    #5.d
    def api_validate(self):
        "Validate the chain"

        if self.chain.is_chain_valid(self.chain.chain):

            response = {'message': 'Chain is valid.',
                        'valid': True}
            return (jsonify(response), 200)

        else:

            response = {'message': 'Chain is not valid!',
                        'valid': False}
            return (jsonify(response), 500)

    #5.e
    def api_overwrite_block(self):
        "Overwrite the third block's record field with a new profile record generated using faker library."

        if len(self.chain.chain) < 3:
            response = {'message': 'There are less than 3 blocks in the chain.'}
            return (jsonify(response), 500)

        # create a new profile record using the faker library
        fake = Faker(locale='en_US')
        # new_record = {
        #     'name': 'Fake',
        #     'address': 'Fake',
        #     'email': 'Fake',
        #     'phone_number': 'Fake',
        #     'job_title': 'Fake',
        #     'gender': 'Fake',
        # }
        new_record = fake.simple_profile()

        # update the record field of the 3rd block in the chain with the new record
        blockchain_app.chain.chain[2]['record'] = new_record
        response = {'message': '3rd record with index:2 has been updated with fake details'}

        return (jsonify(response), 200)



    def api_shutdown(self):
        "Shutdown the Flask WebApp"

        # depricated
        # request.environ.get('werkzeug.server.shutdown')()

        return jsonify({'message': 'Shutting down'}), 200

    def start(self):
        "Start the Flask-based Blockchain WebApp."

        self.thread.start()

    def stop(self):
        "Stop the Flask-based Blockchain WebApp."

        if self.thread.is_alive():

            return requests.get(f'{self.host_url}/shutdown')

In [ ]:
import requests
import json
import time

with BlockchainApp() as blockchain_app:

    # wait for server thread ready
    time.sleep(5)
    
    print("-------------------------------------------------#5.a-------------------------------------------------")
    # mine 20 blocks
    for _ in range(20):
        response = requests.post(f'{blockchain_app.host_url}/blocks')

    # get the current blockchain
    response = requests.get(f'{blockchain_app.host_url}/blocks')

    # print the blockchain
    print(json.dumps(response.json(), indent=2))
    print("-------------------------------------------------#5.b-------------------------------------------------")
    # get male names from the blockchain
    male_names_response = requests.get(f'{blockchain_app.host_url}/male_names')
    male_names = male_names_response.json()['male_names']
    
    # print male names
    print("Male Names:")
    for name in male_names:
        print(name)
    print("-------------------------------------------------#5.c-------------------------------------------------")
    # get male names from the blockchain
    female_names_response = requests.get(f'{blockchain_app.host_url}/female_names')
    female_names = female_names_response.json()['female_names']
    
    # print female names
    print("Female usernames:")
    for name in female_names:
        print(name)
    print("-------------------------------------------------Validating before alteration-------------------------------------------------")   
    response = requests.get(f'{blockchain_app.host_url}/blocks/validate') 
    if response.json()['valid']:
        print('Blockchain is valid.')
    else:
        print('Blockchain is not valid.')
    print("-------------------------------------------------Alteration-------------------------------------------------")   
    response = requests.post(f'{blockchain_app.host_url}/overwrite_block')
    print("Chain={}".format(json.dumps(response.json(), indent=2)))
    print("-------------------------------------------------Printing current Blockchain-------------------------------------------------")   
    # get the current blockchain
    response = requests.get(f'{blockchain_app.host_url}/blocks')
    # print the blockchain
    print(json.dumps(response.json(), indent=2))
    print("-------------------------------------------------Validating after alteration-------------------------------------------------")   
    response = requests.get(f'{blockchain_app.host_url}/blocks/validate') 
    if response.json()['valid']:
        print('Blockchain is valid.')
    else:
        print('Blockchain is not valid.')

INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://localhost:5000
INFO:werkzeug:Press CTRL+C to quit
ERROR:__main__:Exception on /blocks [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/flask/app.py", line 2528, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.9/dist-packages/flask/app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.9/dist-packages/flask/app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.9/dist-packages/flask/app.py", line 1799, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "<ipython-input-2-6ff8e32e13ae>", line 87, in api_blocks
    prev_hash = self.chain.hash_block(prev_block)
  File "<ipython-input-1-f1332513ca49

-------------------------------------------------#5.a-------------------------------------------------


ERROR:__main__:Exception on /blocks [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/flask/app.py", line 2528, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.9/dist-packages/flask/app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.9/dist-packages/flask/app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.9/dist-packages/flask/app.py", line 1799, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "<ipython-input-2-6ff8e32e13ae>", line 87, in api_blocks
    prev_hash = self.chain.hash_block(prev_block)
  File "<ipython-input-1-f1332513ca49>", line 45, in hash_block
    encoded_block = json.dumps(block, sort_keys=True).encode()
  File "/usr/lib/python3.9/json/__init__.py", line 234, in dumps
    return cls(
  File "/usr/lib/python3.9/json/e

{
  "chain": [
    {
      "index": 0,
      "previous_hash": "0",
      "proof": 1,
      "record": {
        "address": "8295 Murphy Pike\nEast Melissa, ND 27327",
        "birthdate": "Sat, 26 Nov 1988 00:00:00 GMT",
        "mail": "danielmoran@hotmail.com",
        "name": "Cheyenne Martin",
        "sex": "F",
        "username": "autumn62"
      },
      "timestamp": "2023-04-11 22:05:49.793048"
    }
  ],
  "length": 1
}
-------------------------------------------------#5.b-------------------------------------------------
Male Names:
-------------------------------------------------#5.c-------------------------------------------------
Female usernames:
autumn62
-------------------------------------------------Validating before alteration-------------------------------------------------
Blockchain is valid.
-------------------------------------------------Alteration-------------------------------------------------
Chain={
  "message": "There are less than 3 blocks in the chain."

In [ ]:
import time

with BlockchainApp() as blockchain_app:

    # wait for server thread ready
    time.sleep(5)

    for _ in range(4):

        response = requests.post(f'{blockchain_app.host_url}/blocks')
        print("Proof={}".format(response.json()['block']['proof']))

    response = requests.get(f'{blockchain_app.host_url}/blocks')
    print("Chain={}".format(json.dumps(response.json(), indent=2)))

    for i in enumerate(blockchain_app.chain.chain):
      print(i)

    original = blockchain_app.chain.chain[1]['proof']
    blockchain_app.chain.chain[1]['proof'] = 12345
    print('Proofs: Original={}, Forged={}'.format(original,blockchain_app.chain.chain[1]['proof']))
    
    response = requests.get(f'{blockchain_app.host_url}/blocks')
    print(json.dumps(response.json(), indent=2))
    
    response = requests.get(f'{blockchain_app.host_url}/blocks/validate')
    print(json.dumps(response.json(), indent=2))
    
    proof_1 = blockchain_app.chain.chain[1]['proof']
    proof_2 = blockchain_app.chain.chain[2]['proof']

    forged_hash = blockchain_app.chain.hash_proof(proof_1, proof_2)

    print("Hashes: \n\tOriginal =\t{}\n\tForged =\t{}".format(blockchain_app.chain.chain[2]['previous_hash'],
                                                              forged_hash))


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.
ERROR:__main__:Exception on /blocks [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/flask/app.py", line 2528, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.9/dist-packages/flask/app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.9/dist-packages/flask/app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.9/dist-packages/flask/app.py", line 1799, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "<ipython-input-2-6ff8e32e13ae>", line 87, in api_blocks
    prev_hash = self.chain.hash_block(prev_block)
  File "<ipython-input-1-f1332513ca49>", line 45, in hash_block
    encoded_block = json.dumps(b

JSONDecodeError: ignored